In [1]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

Satelli sampling means variance based sampling, but not a good measure for non-uniform distribution. Advantage: just extends series further if you want to increase number of samples
Parallelization of sampling --> batch run (concurrent)

# Added modules

In [ ]:
def get_theta(x, mu, sigma):
    """
    x: fraction of similar neighbours
    mu: optimal fraction of similar neighbours
    sigma: acceptance range
    """
    theta = np.exp(-((x - mu) ** 2) / (2 * sigma ** 2))
    return theta

In [ ]:
def gaussian_function(x, mu, sigma):
    return np.exp(-((x - mu) ** 2) / (2 * sigma ** 2))

# Example usage:
x_values = np.linspace(0, 1, 100)
mu = 0.5  # Peak in the middle
sigma = 0.6  # Controls the width

y_values = gaussian_function(x_values, mu, sigma)

# You can plot the function to visualize it
import matplotlib.pyplot as plt

plt.plot(x_values, y_values)
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title(f'Gaussian Function with Peak at {mu}')
plt.show()

# Experiment

In [ ]:
from SALib.sample import saltelli
from SALib.analyze import sobol

In [2]:
import modules as modules
import model as model
from model import Schelling

def schelling_singapore(num_runs, num_steps, burn_in_period, minority_pc, property_value_weight, alpha, mu_theta, sigma_theta, density, policy = True):

    # initialize storage for parameters
    overall_desirability_entropy = []
    overall_agent_entropy = []
    overall_utility = []

    # run the model several times for the same parameter setting 
    for run in range(num_runs):
        print('I am in round', run)
        
        # initialize model
        models = Schelling(
            property_value_func=modules.property_value_quadrants,
            income_func=modules.income_func,
            desirability_func=modules.desirability_func,
            utility_func=modules.utility_func,
            price_func=modules.price_func,
            compute_similar_neighbours=modules.compute_similar_neighbours,
            calculate_gi_star = modules.calculate_gi_star,
            update_interested_agents_func = modules.update_interested_agents_concurrently,
            price_func_cap=modules.price_func_cap,
            policy_singapore=policy,
            #height=20,
            #width=20,
            radius=1,
            density=density,
            minority_pc=minority_pc,
            alpha=alpha,
            income_scale=1.5, # the scale by which the income is higher than the property value
            property_value_weight=property_value_weight,
            mu_theta = mu_theta,
            sigma_theta = sigma_theta,
            seed=42)

        # Run the model for a certain number of steps
        for _ in range(num_steps):
            models.step()

        # call necessary data collectors
        agent_data = models.datacollector.get_agent_vars_dataframe()
        model_data_entropy = models.datacollector.get_model_vars_dataframe()
    
        # Compute mean and standard deviation of entropies over time per run
        desirability_entropy = model_data_entropy['Desirability entropy'].to_numpy()
        # desirability_entropy_std = model_data_entropy['Desirability entropy'].std()
        mean_desirability_entropy = np.mean(np.array(desirability_entropy[burn_in_period::]))
        
        agent_entropy = model_data_entropy['Agent entropy'].to_numpy()
        # agent_entropy_std = model_data_entropy['Agent entropy'].std()
        mean_agent_entropy = np.mean(np.array(agent_entropy[burn_in_period::]))
        
        
        # Compute mean and standard deviation of utility per time step over each agent 
        utility_mean = agent_data.groupby(level='Step')['Utility'].mean()
        utility_std = agent_data.groupby(level='Step')['Utility'].std()
        
        # Compute mean and standard deviation of utility over time 
        utility_mean = pd.DataFrame(utility_mean).to_numpy()
        utility_std = pd.DataFrame(utility_std).to_numpy()
        mean_utility_interim = np.mean(utility_mean[burn_in_period::])

        # Append all results to storage
        overall_desirability_entropy.append(mean_desirability_entropy)
        overall_agent_entropy.append(mean_agent_entropy)
        overall_utility.append(mean_utility_interim)

    return np.array(overall_desirability_entropy), np.array(overall_agent_entropy), np.array(overall_utility)

In [ ]:
# experiment settings
num_iters = 100
burn_in_period = 5
model_steps = 30

# model settings
density = 0.8
minority = 0.3
alpha = 0.5
property_value_weight = 0.1
mu = 0.8
sigma = 0.5

# Run simulation
singapore_policy_effect = schelling_singapore(num_iters, model_steps, burn_in_period, density, minority, alpha, property_value_weight, mu, sigma)

In [ ]:
# Save simulation 
singapore_initial_results = np.column_stack((singapore_policy_effect[0], singapore_policy_effect[1], singapore_policy_effect[2]))

# Save the stacked array to a CSV file
np.savetxt('singapore_policy.csv', singapore_initial_results, delimiter=',', header='desirability_entropy, agent_entropy, utility') 

In [3]:
# experiment settings
num_iters = 100
burn_in_period = 5
model_steps = 30

# model settings
density = 0.8
minority = 0.3
alpha = 0.5
property_value_weight = 0.1
mu = 0.8
sigma = 0.5

# Run simulation
no_policy_effect = schelling_singapore(num_iters, model_steps, burn_in_period, density, minority, alpha, property_value_weight, mu, sigma, policy=False)

I am in round 0
I am in round 1
I am in round 2
I am in round 3
I am in round 4
I am in round 5
I am in round 6
I am in round 7
I am in round 8
I am in round 9
I am in round 10
I am in round 11
I am in round 12
I am in round 13
I am in round 14
I am in round 15
I am in round 16
I am in round 17
I am in round 18
I am in round 19
I am in round 20
I am in round 21
I am in round 22
I am in round 23
I am in round 24
I am in round 25
I am in round 26
I am in round 27
I am in round 28
I am in round 29
I am in round 30
I am in round 31
I am in round 32
I am in round 33
I am in round 34
I am in round 35
I am in round 36
I am in round 37
I am in round 38
I am in round 39
I am in round 40
I am in round 41
I am in round 42
I am in round 43
I am in round 44
I am in round 45
I am in round 46
I am in round 47
I am in round 48
I am in round 49
I am in round 50
I am in round 51
I am in round 52
I am in round 53
I am in round 54
I am in round 55
I am in round 56
I am in round 57
I am in round 58
I am in

In [4]:
no_policy_results = np.column_stack((no_policy_effect[0], no_policy_effect[1], no_policy_effect[2]))

# Save the stacked array to a CSV file
np.savetxt('no_policy_result.csv', no_policy_results, delimiter=',', header='desirability_entropy, agent_entropy, utility') 

# Plots for initial experiment

In [ ]:
singapore_results = pd.read_csv('singapore_policy.csv')
vienna_results = pd.read_csv('vienna_policy.csv')
random_results = pd.read_csv('no_policy_result.csv')

In [ ]:
desirability_vienna = vienna_results['# desirability_entropy'].to_numpy()
desirability_singapore = singapore_results['# desirability_entropy'].to_numpy()
desirability_random = random_results['# desirability_entropy'].to_numpy()

agent_vienna = vienna_results[' agent_entropy'].to_numpy()
agent_singapore = singapore_results[' agent_entropy'].to_numpy()
agent_random = random_results[' agent_entropy'].to_numpy()

utility_vienna = vienna_results[' utility'].to_numpy()
utility_singapore = singapore_results[' utility'].to_numpy()
utility_random = random_results[' utility'].to_numpy()

plt.figure(dpi = 300, figsize = (10, 16))

plt.subplot(311)
# Create distribution plots for both datasets
sns.histplot(desirability_vienna, kde=True, color='#63d298ff', label='Policy: Vienna', stat="density")
sns.histplot(desirability_singapore, kde=True, color='#fff176ff', label='Policy: Singapore', stat="density")
sns.histplot(desirability_random, kde=True, color='#ff5252ff', label='no policy', stat="density")

# Customize the plot
plt.title('Desirability: Distribution Plots of different Policies')
plt.xlabel('Value')
plt.ylabel('Density')
plt.legend()

plt.subplot(312)
# Create distribution plots for both datasets
sns.histplot(agent_vienna, kde=True, color='#63d298ff', label='Policy: Vienna', stat="density")
sns.histplot(agent_singapore, kde=True, color='#fff176ff', label='Policy: Singapore', stat="density")
sns.histplot(agent_random, kde=True, color='#ff5252ff', label='no policy', stat="density")

# Customize the plot
plt.title('Agent entropy: Distribution Plots of different Policies')
plt.xlabel('Value')
plt.ylabel('Density')

plt.subplot(313)
# Create distribution plots for both datasets
sns.histplot(utility_vienna, kde=True, color='#63d298ff', label='Policy: Vienna', stat="density")
sns.histplot(utility_singapore, kde=True, color='#fff176ff', label='Policy: Singapore', stat="density")
sns.histplot(utility_random, kde=True, color='#ff5252ff', label='no policy', stat="density")

# Customize the plot
plt.title('Average utility: Distribution Plots of different Policies')
plt.xlabel('Value')
plt.ylabel('Density')

# Show the plot
plt.show()
plt.close()

# Variations of relevant parameters for experiment